In [1]:
import numpy as np
import pandas as pd
#import cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os


from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed, SpatialDropout1D
from keras.models import Model
from keras.layers.normalization import BatchNormalization

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers, regularizers, constraints

MAX_NB_WORDS = 80000
MAX_SEQUENCE_LENGTH = 400
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 100

data_train = pd.read_csv('/Users/venkatakrishnamohansunkara/Desktop/DM/classification_data.csv', sep=',')
print(data_train.shape)
data_train.dropna(inplace=True)

data_train['cat_id'] = 0
for i,row in data_train.iterrows():
    if row['Category'] == 'unrest':
        data_train.set_value(i,'cat_id',0)
    else:
        data_train.set_value(i,'cat_id',1)

/anaconda3/envs/textClassifier/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


(17575, 7)


/anaconda3/envs/textClassifier/lib/python3.6/site-packages/ipykernel_launcher.py:39: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/envs/textClassifier/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [2]:
X_data = data_train['Content']
y_data = data_train['cat_id']
X_data = list(X_data)
y_data = list(y_data)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_data)
X_data = tokenizer.texts_to_sequences(X_data)
#X_data = str(X_data)
X_data = pad_sequences(X_data,
        maxlen=MAX_SEQUENCE_LENGTH,
        padding='post',
        truncating='post',
        dtype='float32')

y_data = np.asarray(y_data)
y_data = to_categorical(np.asarray(y_data))
indices = np.arange(X_data.shape[0])
np.random.shuffle(indices)
X_data = X_data[indices]
y_data = y_data[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_data.shape[0])

x_train = X_data[:-nb_validation_samples]
y_train = y_data[:-nb_validation_samples]
x_val = X_data[-nb_validation_samples:]
y_val = y_data[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set
[6862. 7112.]
[1713. 1780.]


In [9]:
# Code for generating input to a test document
data_test_df = pd.read_csv('document similarity/document_similarity.csv',sep=',')
data_test_content = data_test_df['content']
X_data_test = tokenizer.texts_to_sequences(data_test_content)
X_data_test = pad_sequences(X_data_test,
        maxlen=MAX_SEQUENCE_LENGTH,
        padding='post',
        truncating='post',
        dtype='float32')

In [12]:
GLOVE_DIR = "/Users/venkatakrishnamohansunkara/Desktop/DM/textClassifier"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))

word_index = tokenizer.word_index
# Embedding matrix should consist of all the unique words (which is the lenght of word index) as rows and their embeddings as columns (100 dimensions)
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
         #words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = EMBEDDING_DIM*[0]

Total 400000 word vectors.


In [13]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

    
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        #a = K.exp(ait)
        a = K.softmax(ait)
        # apply mask after the exp. will be re-normalized next
        #if mask is not None:
        #    # Cast the mask to floatX to avoid float64 upcasting in theano
        #    a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [14]:
input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
embedding_layer = Embedding(input_dim=len(word_index)+1,
        output_dim=EMBEDDING_DIM,
        input_length=MAX_SEQUENCE_LENGTH,
        weights=[embedding_matrix],
        embeddings_regularizer=regularizers.l2(0.00),
        trainable=True)(input_layer)
drop1 = SpatialDropout1D(0.3)(embedding_layer)
lstm_1 = Bidirectional(LSTM(128, name='blstm_1',
    activation='tanh',
    recurrent_activation='hard_sigmoid',
    recurrent_dropout=0.0,
    dropout=0.5,
    kernel_initializer='glorot_uniform',
    return_sequences=True),
    merge_mode='concat')(drop1)
lstm_1 = BatchNormalization()(lstm_1)
att_layer = AttentionWithContext()(lstm_1)
drop3 = Dropout(0.5)(att_layer)
predictions = Dense(2, activation='sigmoid')(drop3)
model = Model(inputs=input_layer, outputs=predictions)

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 400)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 400, 100)          10835300  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 400, 100)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 400, 256)          234496    
_________________________________________________________________
batch_normalization_1 (Batch (None, 400, 256)          1024      
_________________________________________________________________
attention_with_context_1 (At (None, 256)               66048     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
__________

In [15]:
model.load_weights('/Users/venkatakrishnamohansunkara/Desktop/DM/model_bi_lstm.weights.h5')

In [51]:
get_layer_output = K.function([model.layers[0].input, K.learning_phase()], [model.layers[4].output])
out = get_layer_output([x_train[4567:4568],0])  # test mode
print(out[0].shape)

(1, 400, 256)


In [52]:
eij = np.tanh(np.dot(out[0],model.layers[5].get_weights()[0]))
t = np.dot(eij,model.layers[5].get_weights()[2])
ai = np.exp(t)
weights = ai/np.sum(ai)

In [53]:
weights.shape

(1, 400)

In [56]:
id2word = {v: k for k, v in word_index.items()}
with open("visualization_bilstm.html", "w") as html_file:
    html_file.write('<!DOCTYPE html>\n')
    html_file.write('<html>\n')
    html_file.write('<body>\n')
    word_no = 0
    for i in x_train[4567]:
        if i!=0:
        # Iterate through the words in the sentence
            word = id2word[i]
            alpha = weights[0][word_no]*10000
            html_file.write('<font style="background-color: rgba(255, 0, 0, %f)">%s</font>\n' %(alpha,word))
            print(word+'-->'+str(alpha)+' ')
            word_no+=1
    html_file.write('</body>\n')
    html_file.write('</html>\n')

gurgaon-->0.8232082473114133 
agitating-->0.52955663704779 
lecturers-->0.23384272935800254 
called-->0.3280951932538301 
off-->0.300654282909818 
their-->0.4169056046521291 
statewide-->0.6617409235332161 
strike-->0.16963982488960028 
on-->0.3416162871872075 
saturday-->0.21803072741022334 
after-->0.17520276742288843 
the-->0.3250337249482982 
director-->0.32167296012630686 
general-->0.4257334876456298 
of-->0.26901743694907054 
higher-->0.23834654712118208 
education-->0.31716808734927326 
dghe-->0.4826814620173536 
promised-->0.22978714696364477 
to-->0.1839655487856362 
look-->0.44891821744386107 
into-->0.4781209645443596 
their-->0.49056121497415006 
demands-->0.47319699660874903 
and-->0.13036752534389962 
address-->0.36598536098608747 
their-->0.27159014280186966 
grievances-->0.5909315586904995 
the-->0.2673931885510683 
development-->0.23472846805816516 
paved-->0.224234663619427 
the-->0.16983452951535583 
way-->0.22595591872232035 
for-->0.2221556133008562 
regular-->0.2

In [25]:
data_test_df['Bi-LSTM embeddings'] = ''

In [28]:
doc_output = K.function([model.layers[0].input, K.learning_phase()], [model.layers[5].output])
out = doc_output([X_data_test,0])
for e,i in enumerate(out[0]):
    data_test_df.set_value(e,'Bi-LSTM embeddings',i)

/anaconda3/envs/textClassifier/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [30]:
data_test_df['bi-lstmcosine'] = ''

In [32]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

In [33]:
for i,row1 in data_test_df.iterrows():
    l= []
    for j,row2 in data_test_df.iterrows():
        cs = cos_sim(row1['Bi-LSTM embeddings'],row2['Bi-LSTM embeddings'])
        l+=[cs]
    data_test_df.set_value(i,'bi-lstmcosine',np.asarray(l))

/anaconda3/envs/textClassifier/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [35]:
for i,row in data_test_df.iterrows():
    print(i);
    r = np.sort(row['bi-lstmcosine'])[::-1][0:3]
    r2 = np.argsort(row['bi-lstmcosine'])[::-1][0:3]
    print(r)
    print(r2)

0
[1.         0.96025157 0.9528559 ]
[0 6 2]
1
[1.        0.9716942 0.9631069]
[1 7 5]
2
[0.9999999 0.9528559 0.9153167]
[2 0 1]
3
[1.         0.98588735 0.96697867]
[3 6 4]
4
[1.         0.98129034 0.97444236]
[4 6 7]
5
[1.        0.9778861 0.9631069]
[5 7 1]
6
[1.         0.98588735 0.98129034]
[6 3 4]
7
[0.99999994 0.9778861  0.97444236]
[7 5 4]
8
[1.         0.79635936 0.7540187 ]
[8 1 5]
